## Number of operations

In [ ]:
import numpy as np
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import pickle 

mnist_tmaxs = [2**exponent for exponent in range(1,6)]
mnist_error = 100 - np.array([89.68999981880188, 98.10999631881714, 99.04999732971191, 99.1599977016449, 99.24999475479126])
mnist_ops = np.array([49, 66, 101, 171, 310])*1e3
mnist = pickle.load(open('../mnist/mnist-results.pkl', 'rb'))

cifar_tmaxs = [2**exponent for exponent in range(3,7)]
cifar_error = 100 - np.array([83.30999612808228, 91.3599967956543, 92.14999675750732, 92.25999712944031])
cifar_ops = np.array([4.356, 6.912, 12.024, 22.249]) * 1e6
cifar = pickle.load(open('../cifar10/cifar10-results.pkl', 'rb'))

imagenet_tmaxs = [2**exponent for exponent in range(4,7)]
imagenet_error = 100 - np.array([54.44, 64.44, 67.24])
imagenet_ops = np.array([537.928, 775.855, 1251.711]) * 1e6
imagenet_times = [146, 290, 578]


df = pd.DataFrame({
    'Dataset': ['MNIST'] * 5 + ['CIFAR10'] * 4 + ['ImageNet'] * 3,
    'Text': mnist_tmaxs + cifar_tmaxs + imagenet_tmaxs,
    '% accuracy error': np.concatenate((mnist_error, cifar_error, imagenet_error)),
    'Ops per frame': np.concatenate((mnist_ops, cifar_ops, imagenet_ops)), 
})

df.loc[0, 'Text'] = "$T_{\mathrm{max}}=\mathrm{2}$"
df.loc[5, 'Text'] = "$T_{\mathrm{max}}=\mathrm{8}$"
df.loc[9, 'Text'] = "$T_{\mathrm{max}}=\mathrm{16}$"

fig = px.line(df, x="Ops per frame", y="% accuracy error", facet_col="Dataset", text="Text")
fig.update_traces(textposition=["middle right", "top right", "top right", "top center", "top center"], textfont=dict(size=11)) # "bottom left", "middle right"

ANN_kws = dict(showlegend=False, mode="markers+text", text=['<b>ANN</b>'], textfont=dict(size=14), textposition="top center", marker=dict(symbol='x', size=13, color='coral'))
fig.add_trace(go.Scatter(x=[0.22584e6], y=[100-99.22999739646912], **ANN_kws), row = 1, col = 1)
fig.add_trace(go.Scatter(x=[171.680256e6], y=[100-92.38999485969543], **ANN_kws), row = 1, col = 2)
fig.add_trace(go.Scatter(x=[7609.416192e6], y=[100-68.11000108718872], **ANN_kws), row = 1, col = 3)

fig.update_xaxes(type="log", matches=None, showticklabels=True)
fig.update_yaxes(matches=None, showticklabels=True)

import plotly.io as pio
pio.kaleido.scope.default_format = "pdf"
pio.kaleido.scope.default_width = 1000
pio.kaleido.scope.default_height = 350

fig.update_layout(
    # autosize=False,
    # width=400,
    # height=300,
    # margin=dict(l=10,r=10,b=10,t=0),
    template="plotly_white"
)

# fig['layout']['margin'] = dict(l=20, t=20)#['t'] = 0
# fig.show()
# fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig.write_image('n_ops.pdf')

# from IPython.display import Image
# img_bytes = fig.to_image(format="png", width=1000, height=350, scale=1)
# Image(img_bytes)

## Accuracy over Tmax

In [ ]:
df = pd.DataFrame({
    'Dataset': ['MNIST'] * 5 + ['CIFAR10'] * 4 + ['ImageNet'] * 3,
    'Tmax per layer': mnist_tmaxs + cifar_tmaxs + imagenet_tmaxs,
    '% accuracy error': np.concatenate((mnist_error, cifar_error, imagenet_error)),
    'Ops per frame': np.concatenate((mnist_ops, cifar_ops, imagenet_ops)), 
})
df

fig1 = px.line(df, x="Tmax per layer", y="% accuracy error", color="Dataset", log_x=True)
fig2 = px.scatter(df, x="Tmax per layer", y="% accuracy error", color="Dataset", log_x=True)
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.update_xaxes(type="log")
fig3.update_layout(
    # autosize=False,
    # width=400,
    # height=300,
    xaxis_title="Tmax per layer", 
    yaxis_title="% accuracy error",
    margin=dict(l=10,r=10,b=50,t=10),
    template="plotly_white",
    # log_x=True,
)
fig3['data'][3]['showlegend']=False
fig3['data'][4]['showlegend']=False
fig3['data'][5]['showlegend']=False

import plotly.io as pio
pio.kaleido.scope.default_format = "pdf"
pio.kaleido.scope.default_width = 800
pio.kaleido.scope.default_height = 350

fig3.write_image('simulation_accs.pdf')
fig3.show()



In [ ]:
mnist = pickle.load(open('../mnist/mnist-results.pkl', 'rb'))
df = pd.DataFrame()

for percentile, data_dict in mnist.items():
    for t_max, metrics in data_dict.items():
        df_temp = pd.DataFrame({
            'Dataset': ['MNIST'],
            'Normalization percentile': [percentile],
            'Tmax': [t_max],
            'Time steps': [metrics['time steps']],
            'Accuracy error': [100 - metrics['acc']],
            'Early spikes': [metrics['early spikes']],
            'Number of operations per frame': [metrics['n_ops']],
            'Neurons': [metrics['n_neurons']],
            'SynOps': [metrics['n_synops']]
        })
        df = pd.concat((df, df_temp))
df = df.reset_index()

fig = px.line(df, x='Tmax', y='Accuracy error', color='Normalization percentile')
fig.update_xaxes(type="log", matches=None, showticklabels=True)
fig.show()


In [ ]:
metrics

In [ ]:
df[df['Tmax'] == 16]

In [ ]:
mnist[98.5][16]

## Energy estimation

In [ ]:
energy_add = lambda b1, b2: max(b1, b2) + abs(b1-b2)/2
energy_mac = lambda b1, b2: b1*b2

mnist_energy = np.array([ops*energy_add(np.log2(tmax), 32) for ops, tmax in zip(mnist_ops, mnist_tmaxs)])
cifar_energy = np.array([ops*energy_add(np.log2(tmax), 32) for ops, tmax in zip(cifar_ops, cifar_tmaxs)])
imagenet_energy = np.array([ops*energy_add(np.log2(tmax), 32) for ops, tmax in zip(imagenet_ops, imagenet_tmaxs)])

df = pd.DataFrame({
    'Dataset': ['MNIST'] * 5 + ['CIFAR10'] * 4 + ['ImageNet'] * 3,
    'Text': mnist_tmaxs + cifar_tmaxs + imagenet_tmaxs,
    '% accuracy error': np.concatenate((mnist_error, cifar_error, imagenet_error)),
    'Relative dynamic energy per frame': np.concatenate((mnist_energy/0.22584e6/energy_mac(32,8), cifar_energy/171.680256e6/energy_mac(32,8), imagenet_energy/7609.416192e6/energy_mac(32,8))), 
})

df.loc[0, 'Text'] = "$T_{\mathrm{max}}=\mathrm{2}$"
df.loc[5, 'Text'] = "$T_{\mathrm{max}}=\mathrm{8}$"
df.loc[9, 'Text'] = "$T_{\mathrm{max}}=\mathrm{16}$"

fig = px.line(df, x="Relative dynamic energy per frame", y="% accuracy error", facet_col="Dataset", text="Text")
fig.update_traces(textposition=["middle right", "top right", "top right", "top right", "top right"], textfont=dict(size=11)) # "bottom left", "middle right"

ANN_kws = dict(showlegend=False, mode="markers+text", text=['<b>ANN 32bit</b>          '], textfont=dict(size=14), textposition="top center", marker=dict(symbol='x', size=13, color='coral'))
fig.add_trace(go.Scatter(x=[1], y=[100-99.22999739646912], **ANN_kws), row = 1, col = 1)
fig.add_trace(go.Scatter(x=[1], y=[100-92.38999485969543], **ANN_kws), row = 1, col = 2)
fig.add_trace(go.Scatter(x=[1], y=[100-68.11000108718872], **ANN_kws), row = 1, col = 3)

fig.update_xaxes(type="log", matches=None, showticklabels=True)
fig.update_yaxes(matches=None, showticklabels=True)


fig.update_layout(
    # autosize=False,
    # width=400,
    # height=300,
    # margin=dict(l=50,r=20,b=20,t=0),
    template="plotly_white"
)

import plotly.io as pio
pio.kaleido.scope.default_format = "pdf"
pio.kaleido.scope.default_width = 1000
pio.kaleido.scope.default_height = 350

fig.write_image('energy.pdf')
